Цель - провести анализ тональности базы данных ресторана с испоьзованием 
методологии NPL

# Предобработка

the organization of the data is carried out - Выпольнается организация данных.

In [1]:
#instalation of libraries - установка библиотек
import sys
import subprocess
subprocess.run([sys.executable, "-m", "pip", "install", "pandas", "numpy", "scikit-learn"])


CompletedProcess(args=['c:\\Users\\Usuario\\Videos\\Innovation Web\\Project\\analis_emotion\\.venv\\Scripts\\python.exe', '-m', 'pip', 'install', 'pandas', 'numpy', 'scikit-learn'], returncode=0)

In [2]:
import pandas as pd
import numpy as np


print(f"Pandas {pd.__version__} y NumPy {np.__version__} установлены правильно ") 

Pandas 2.3.3 y NumPy 2.3.5 установлены правильно 


In [5]:
import os


df = pd.read_csv('../data/raw/Test_data_marked.csv', header=None, encoding='utf-8')

print("\n=== Date ===")
print(df.head(3))


=== Date ===
              0              1             2             3       4        5  \
0  Общая оценка  Цена/качество  Расположение  Качество сна  Номера  Чистота   
1             5              5             5           NaN       5        5   
2             5              5             5             5       5        5   

              6                                                  7  
0  Обслуживание                                       Текст отзыва  
1             5  Элегантный отель в самом центре Дубай. Красивы...  
2             5  Если стиль вашей поездки в Дубай-- это шоппинг...  


In [6]:
column_text = df[7]
print(f"Númber of text: {len(column_text)}")

Númber of text: 6877


In [7]:
#удаляем первую строку
text_cleaning = column_text .iloc[1:].reset_index(drop=True) 
print(text_cleaning.head())

0    Элегантный отель в самом центре Дубай. Красивы...
1    Если стиль вашей поездки в Дубай-- это шоппинг...
2    Провели с супругой там уик енд. Брали самый лю...
3    По дороге домой для того, чтобы перекантоватьс...
4    меня впечатлили размеры! самый огромный номер,...
Name: 7, dtype: object


In [9]:
text_cleaning = text_cleaning.apply(lambda x: str(x).strip() if pd.notnull(x) else "")

# 3. Filtrar solo textos no vacíos
text_cleaning = text_cleaning[text_cleaning.str.len() > 0]

print(f"\n=== after filtering - после фильтрации ===")
print(f"Тексты, пригодные для анализа: {len(text_cleaning)}")

# 4. Crear un DataFrame solo con los textos (para seguir trabajando)
df_textos = pd.DataFrame({'text_russi': text_cleaning})

print("\n=== DATAFRAME CON TEXTOS LIMPIOS ===")
print(df_textos.head())
print(f"\nDimensiones: {df_textos.shape}")


=== after filtering - после фильтрации ===
Тексты, пригодные для анализа: 6876

=== DATAFRAME CON TEXTOS LIMPIOS ===
                                          text_russi
0  Элегантный отель в самом центре Дубай. Красивы...
1  Если стиль вашей поездки в Дубай-- это шоппинг...
2  Провели с супругой там уик енд. Брали самый лю...
3  По дороге домой для того, чтобы перекантоватьс...
4  меня впечатлили размеры! самый огромный номер,...

Dimensiones: (6876, 1)


# Тokenization - Токенизация

Мы испулзуем метод N-gram

Для правильного выполнения токенизации мы загружаем языковые файлы, содержащие грамматические правила и словарный запас. Эти файлы служат базой знаний, чтобы алгоритм мог точно понимать и обрабатывать русский текст."

"To perform tokenization correctly, we download language files containing grammatical rules and vocabulary. These files serve as a knowledge base so the algorithm can accurately understand and process Russian text." 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [10]:
import re
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Descargar los datos necesarios
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Get Russian stopwords / Получить русские стоп-слова

russian_stopwords = set(stopwords.words('russian'))

Function to clean and tokenize

In [12]:
def clean_tokenize(text):
     # Tokenize / Токенизировать
    tokens = word_tokenize(str(text), language='russian')
    # Lowercase and filter / Нижний регистр и фильтрация

    clean_tokens = [word.lower() for word in tokens 
                   if word.lower() not in russian_stopwords 
                   and word.isalpha()]
    return clean_tokens


In [13]:
stop_words = set(stopwords.words('russian'))
pattern = r'https?://\S+|www\.\S+'

def preprocess_text(text):
    # Eliminar URLs
    text = re.sub(pattern, '', str(text))
    
    # Minúsculas
    text = text.lower()
    
    # CORRECCIÓN: Mantener letras rusas y espacios
    text = re.sub(r'[^а-яё\s]', ' ', text)
    
    # Tokenizar
    tokens = word_tokenize(text, language='russian')
    
    # Eliminar stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    # CORRECCIÓN: Unir tokens correctamente
    clean_text = ' '.join(tokens)  # ¡Esto faltaba!
    
    return clean_text

# Probar
test_text = "Элегантный отель в самом центре Дубай. Красивые номера."
result = preprocess_text(test_text)
print("Resultado:", result)

df_textos['tokens'] = df_textos['text_russi'].apply(clean_tokenize)
df_textos['clear_text'] = df_textos['text_russi'].apply(preprocess_text)

total_tokens = sum(len(tokens) for tokens in df_textos['tokens'])
promedio_tokens = total_tokens / len(df_textos)

Resultado: элегантный отель самом центре дубай красивые номера


перемостр результатов - review of results

In [14]:
df_textos['tokens'] = df_textos['text_russi'].apply(clean_tokenize)
print(f"✅ Tokenización completada: {len(df_textos)} reseñas")

✅ Tokenización completada: 6876 reseñas


# Stemming

In [15]:
from nltk.stem.snowball import SnowballStemmer
#Создаем стеммиг для данные 
stemmer = SnowballStemmer("russian")

def stem_tokens(tokens_list):
    """Stemming russian"""
    return [stemmer.stem(token) for token in tokens_list]

#aplicamos toda la columna tokens del dataFrame
df_textos['tokens_stemmed'] = df_textos['tokens'].apply(stem_tokens)

print("✅ Stemming aplicado a", len(df_textos), "reseñas")
print("\nEjemplo de resultados:")
print("Original:", df_textos['tokens'].iloc[0][:8])
print("Stemmed:", df_textos['tokens_stemmed'].iloc[0][:8])




✅ Stemming aplicado a 6876 reseñas

Ejemplo de resultados:
Original: ['элегантный', 'отель', 'самом', 'центре', 'дубай', 'красивые', 'стильные', 'номера']
Stemmed: ['элегантн', 'отел', 'сам', 'центр', 'дуба', 'красив', 'стильн', 'номер']


In [16]:
df_textos['texto_stemmed'] = df_textos['tokens_stemmed'].apply(lambda x: ' '.join(x))

print("\n" + "="*50)
print("ESTADO ACTUAL: PREPROCESAMIENTO COMPLETADO")
print("="*50)
print(f"DataFrame tiene {len(df_textos)} filas")
print("Columnas disponibles:", df_textos.columns.tolist())


ESTADO ACTUAL: PREPROCESAMIENTO COMPLETADO
DataFrame tiene 6876 filas
Columnas disponibles: ['text_russi', 'tokens', 'clear_text', 'tokens_stemmed', 'texto_stemmed']


Cargamos datos positivos y negative, que serviran para entrenar la data

In [17]:
# positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
# positive['sentiment'] = 'positive'

# negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
# negative['sentiment'] = 'negative'

# df_labeled = pd.concat([positive, negative], ignore_index=True)


# N-grams

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
# 1. Preparar datos: unir tokens stemmeados en texto
df_textos['texto_stemmed'] = df_textos['tokens_stemmed'].apply(lambda x: ' '.join(x))

# 2. Crear vectorizador con unigramas y bigramas
vectorizador = CountVectorizer(ngram_range=(1, 2))
X_ngrams = vectorizador.fit_transform(df_textos['texto_stemmed'])


In [19]:
print(f"✅ MATRIZ N-GRAMS: {X_ngrams.shape[0]} documentos × {X_ngrams.shape[1]} n-grams")
print(f"📊 Total n-grams únicos: {X_ngrams.shape[1]}")
print(f"🔤 Unigramas: {sum(1 for x in vectorizador.get_feature_names_out() if len(x.split()) == 1)}")
print(f"🔤 Bigramas: {sum(1 for x in vectorizador.get_feature_names_out() if len(x.split()) == 2)}")

✅ MATRIZ N-GRAMS: 6876 documentos × 236417 n-grams
📊 Total n-grams únicos: 236417
🔤 Unigramas: 18487
🔤 Bigramas: 217930


# Vectorization - Векторизация

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Crear vectorizador TF-IDF con unigramas y bigramas
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),      # Unigramas y bigramas
    min_df=2,                # Ignorar términos en menos de 2 documentos
    max_df=0.95,             # Ignorar términos en más del 95% de documentos
    use_idf=True             # Usar Inverse Document Frequency
)

# 2. Aplicar TF-IDF a los textos stemmeados
X_tfidf = tfidf_vectorizer.fit_transform(df_textos['texto_stemmed'])

In [28]:

# Load your cleaned hotel data
#df_hotel = pd.read_csv('../data/raw/Test_data_marked.csv')  # Adjust filename

# Create dummy labels (positive/negative) -
np.random.seed(42)
labels = np.random.choice(['positive', 'negative'], size=len(df), p=[0.5, 0.5])
df['label'] = labels

# Use the clean text column (adjust column name)
df = pd.DataFrame({
    'text': df_textos['clear_text'],  # Your cleaned text column
    'label': df['label']
})

print(f"Dataset size: {len(df)}")
print(df['label'].value_counts())
print(df)

Dataset size: 6877
label
positive    3455
negative    3422
Name: count, dtype: int64
                                                   text     label
0     элегантный отель самом центре дубай красивые с...  positive
1     стиль вашей поездки дубай это шоппинг дубай мо...  negative
2     провели супругой уик енд брали самый люкс видо...  negative
3     дороге домой перекантоваться время пересадки р...  negative
4     впечатлили размеры самый огромный номер которо...  positive
...                                                 ...       ...
6872  очень понравился отель хорошие номера кухней в...  negative
6873  петербурге бываю очень часто поэтому появление...  positive
6874  изумительное место просто ожидала такого понра...  negative
6875  останавливались отеле августе решили провести ...  positive
6876                                                NaN  positive

[6877 rows x 2 columns]


Split Data into Train/Test Sets

In [33]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df['text'], 
    df['label'],
    test_size=0.25,
    random_state=42,
    stratify=df['label']  # Keep class balance
)
x_test = x_test.fillna('') 
print(f"Train size: {len(x_train)}, Test size: {len(x_test)}")
print("Train labels:", y_train.value_counts())

Train size: 5157, Test size: 1720
Train labels: label
positive    2591
negative    2566
Name: count, dtype: int64


Baseline - CountVectorizer with Unigrams

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Create vectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1))

# Transform training data
x_train_vectorized = vectorizer.fit_transform(x_train)

# Train model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(x_train_vectorized, y_train)

# Transform test data and predict
x_test_vectorized = vectorizer.transform(x_test)
predictions = model.predict(x_test_vectorized)

# Results
print("=== Baseline: CountVectorizer (Unigrams) ===")
print(classification_report(y_test, predictions))

=== Baseline: CountVectorizer (Unigrams) ===
              precision    recall  f1-score   support

    negative       0.48      0.47      0.48       856
    positive       0.49      0.50      0.49       864

    accuracy                           0.49      1720
   macro avg       0.49      0.49      0.49      1720
weighted avg       0.49      0.49      0.49      1720



TF-IDF Vectorization (Unigrams)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

# Transform and train
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
model_tfidf = LogisticRegression(random_state=42, max_iter=1000)
model_tfidf.fit(x_train_tfidf, y_train)

# Predict and evaluate
x_test_tfidf = tfidf_vectorizer.transform(x_test)
predictions_tfidf = model_tfidf.predict(x_test_tfidf)

print("=== TF-IDF (Unigrams) ===")
print(classification_report(y_test, predictions_tfidf))

=== TF-IDF (Unigrams) ===
              precision    recall  f1-score   support

    negative       0.49      0.47      0.48       856
    positive       0.49      0.51      0.50       864

    accuracy                           0.49      1720
   macro avg       0.49      0.49      0.49      1720
weighted avg       0.49      0.49      0.49      1720



: TF-IDF with Bigrams and Trigrams

In [38]:
# Test different n-gram ranges
for ngram_range in [(2, 2), (3, 3), (5,5)]:  # Bigrams, then Trigrams
    # Create vectorizer
    tfidf_ngram = TfidfVectorizer(ngram_range=ngram_range)
    
    # Train
    x_train_ngram = tfidf_ngram.fit_transform(x_train)
    model_ngram = LogisticRegression(random_state=42, max_iter=1000)
    model_ngram.fit(x_train_ngram, y_train)
    
    # Predict
    x_test_ngram = tfidf_ngram.transform(x_test)
    predictions_ngram = model_ngram.predict(x_test_ngram)
    
    print(f"=== TF-IDF {ngram_range} ===")
    print(classification_report(y_test, predictions_ngram))

=== TF-IDF (2, 2) ===
              precision    recall  f1-score   support

    negative       0.51      0.45      0.48       856
    positive       0.51      0.58      0.54       864

    accuracy                           0.51      1720
   macro avg       0.51      0.51      0.51      1720
weighted avg       0.51      0.51      0.51      1720

=== TF-IDF (3, 3) ===
              precision    recall  f1-score   support

    negative       0.51      0.40      0.45       856
    positive       0.51      0.62      0.56       864

    accuracy                           0.51      1720
   macro avg       0.51      0.51      0.50      1720
weighted avg       0.51      0.51      0.50      1720

=== TF-IDF (5, 5) ===
              precision    recall  f1-score   support

    negative       0.45      0.02      0.04       856
    positive       0.50      0.97      0.66       864

    accuracy                           0.50      1720
   macro avg       0.48      0.50      0.35      1720
weighted

: XGBoost Classifier

In [41]:
# First install if needed: !pip install xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Convert labels to numbers
label_encoder = LabelEncoder()
y_train_num = label_encoder.fit_transform(y_train)  # negative=0, positive=1
y_test_num = label_encoder.transform(y_test)

# Train XGBoost on TF-IDF features (unigrams)
xgb_model = XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=3,
    gamma=0.2,
    subsample=0.6,
    colsample_bytree=1.0,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27
)

# Use TF-IDF unigrams from Step 4
xgb_model.fit(x_train_tfidf, y_train_num)

# Predict and convert back to text labels
xgb_predictions_num = xgb_model.predict(x_test_tfidf)
xgb_predictions = label_encoder.inverse_transform(xgb_predictions_num)

print("=== XGBoost Classifier ===")
print(classification_report(y_test, xgb_predictions))

=== XGBoost Classifier ===
              precision    recall  f1-score   support

    negative       0.49      0.46      0.48       856
    positive       0.50      0.53      0.51       864

    accuracy                           0.50      1720
   macro avg       0.50      0.50      0.49      1720
weighted avg       0.50      0.50      0.49      1720



Alternative Classifier - Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

rf_model.fit(x_train_tfidf, y_train)
rf_predictions = rf_model.predict(x_test_tfidf)

print("=== Random Forest Classifier ===")
print(classification_report(y_test, rf_predictions))

=== Random Forest Classifier ===
              precision    recall  f1-score   support

    negative       0.50      0.36      0.42       856
    positive       0.50      0.63      0.56       864

    accuracy                           0.50      1720
   macro avg       0.50      0.50      0.49      1720
weighted avg       0.50      0.50      0.49      1720



# Compare All Results

In [53]:
print("\n" + "="*50)
print("COMPARISON OF ALL MODELS")
print("="*50)

# Collect all predictions
models = {
    "Baseline (Unigrams)": predictions,
    "TF-IDF (Unigrams)": predictions_tfidf,
    #"TF-IDF (Bigrams)": predictions_ngram_2,
    "TF-IDF (Trigrams)": predictions_ngram,
    "XGBoost": xgb_predictions,
    "Random Forest": rf_predictions
}

# Simple comparison table
print(f"{'Model':<25} {'Accuracy':<10} {'F1-Score':<10}")
print("-"*45)

for name, pred in models.items():
    # Calculate accuracy
    accuracy = (pred == y_test).mean()
    
    # Calculate F1-score (average of both classes)
    from sklearn.metrics import f1_score
    f1 = f1_score(y_test, pred, average='macro')
    
    print(f"{name:<25} {accuracy:.4f}     {f1:.4f}")


COMPARISON OF ALL MODELS
Model                     Accuracy   F1-Score  
---------------------------------------------
Baseline (Unigrams)       0.4951     0.4949
TF-IDF (Unigrams)         0.4939     0.4934
TF-IDF (Trigrams)         0.5177     0.5130
XGBoost                   0.5020     0.5014
Random Forest             0.4846     0.4777
